In [ ]:
import pandas as pd                                                                                                                                                               
from sklearn.compose import ColumnTransformer                                     
from sklearn.preprocessing import StandardScaler, OneHotEncoder                                                  
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error 
from sklearn.linear_model import Lasso
from scipy.stats import loguniform                               

In [ ]:
dados = pd.read_table("../datasets/Potencial_Novos_Clientes.txt",sep="\t",header=0)

#### Modelo

In [ ]:
# Variáveis explicativas quantitativas (deixar vazio [] caso não haja nenhuma)
lista_X_quanti = ['IDADE_CLIENTE',
                  'RENDA_MENSAL_CLIENTE',
                  'BEHAVIOUR_SCORE_CLIENTE',
                  'QTD_TRANSACOES_3M',
                  'QTD_ITENS_3M',
                  'VALOR_GASTO_3M',
                  'TICKET_MEDIO_3M']

# Variáveis explicativas qualitativas (deixar vazio [] caso não haja nenhuma)
lista_X_quali = ['FLAG_ELETRONICOS_3M',
                 'SATISFACAO_ULTIMA_COMPRA']

In [ ]:
y = dados['VALOR_GASTO_PROX_12M']
X = dados[lista_X_quanti + lista_X_quali]

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y,
                                                        test_size=0.2,
                                                        random_state=123)

In [ ]:
preprocessador = ColumnTransformer(transformers=[
    ("quanti", StandardScaler(), lista_X_quanti),
    ("quali", OneHotEncoder(sparse_output=False, drop="first", handle_unknown='ignore'), lista_X_quali)
])

In [ ]:
X_treino_tratada = preprocessador.fit_transform(X_treino)  # O pré-processamento não deve envolver o conjunto de teste externo, que deve ficar reservado apenas para aplicações de resultados já obtidos, e nunca construções
if lista_X_quali:
    nomes_quali = list(preprocessador.named_transformers_['quali'].get_feature_names_out(lista_X_quali))
else:
    nomes_quali = []
nomes_variaveis = list(lista_X_quanti) + nomes_quali

In [ ]:
X_treino_tratada = pd.DataFrame(X_treino_tratada, columns=nomes_variaveis)
X_treino_tratada.head()

In [ ]:
X_teste_tratada = preprocessador.transform(X_teste)  # Note que o mesmo objeto de pré-processamento deve ser utilizado, agora com 'transform' em vez de 'fit_transform'
X_teste_tratada = pd.DataFrame(X_teste_tratada, columns=nomes_variaveis)
X_teste_tratada.head()

In [ ]:
model = Lasso(random_state=123, max_iter=1000)
model.fit(X_treino_tratada, y_treino)
y_hat_teste = model.predict(X_teste_tratada)
erro = root_mean_squared_error(y_teste, y_hat_teste)

#### Serialização

In [ ]:
import os
import pickle
import glob
from datetime import datetime

In [ ]:
model_folder = '../assets/models/trained/'
filename = f'model_{datetime.now().date()}.pkl'
pickle.dump(model, open(model_folder+filename, 'wb'))

In [ ]:
models = sorted(
    glob.iglob(os.path.join(os.path.dirname(model_folder), '*')),
      key=os.path.getctime, reverse=True) 
print(models)

In [ ]:
load_model = pickle.load(open(models[0], 'rb'))
predictions = load_model.predict(X_teste_tratada)
predictions